<a href="https://colab.research.google.com/github/jwkanggist/EverybodyTensorflow2.0/blob/master/lab21_basic_rnn_mnist_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB21: Basic RNN for MNIST classification
- Train a basic RNN for MNIST classification (seq2vec)
- A 28x28 MNIST image is represented as n_input x n_steps
- The output layer is connected to the final state of the RNN


In [0]:
# preprocessor parts
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt

In [3]:
 # for Tensorboard use
  LOG_DIR = 'drive/data/tb_logs'
	
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip
	
  import os
  if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)
	  
  get_ipython().system_raw(
      'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
      .format(LOG_DIR))
	
  get_ipython().system_raw('./ngrok http 6006 &')
	
  !curl -s http://localhost:4040/api/tunnels | python3 -c \
      "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-01-18 15:54:29--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.204.140.35, 35.153.11.252, 52.3.157.51, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.204.140.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  14.0MB/s    in 0.9s    

2020-01-18 15:54:35 (14.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://c40e0dd7.ngrok.io


In [6]:
# dataset loading part
# 데이터 파이프라인 부분 
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

input_shape = x_train.shape[1]
print(x_train.shape)
print(input_shape)
# [batch, 28,28]

(60000, 28, 28)
28


In [0]:
# model building and training setting part
#  Y = f(X ; W)
dropout_rate=0.1

net_in = tf.keras.layers.Input(shape=(input_shape,input_shape))
net = tf.keras.layers.SimpleRNN(units=150,activation='tanh',
                                    return_sequences=True,
                                    return_state=False)(net_in)
net = tf.keras.layers.SimpleRNN(units=150,activation='tanh',
                                    return_sequences=True,
                                    return_state=False)(net)
net = tf.keras.layers.GlobalAvgPool1D()(net) # shape = [ batch_size, feature]
net = tf.keras.layers.Dropout(dropout_rate)(net)
net = tf.keras.layers.Dense(units=50, activation='relu')(net)
net = tf.keras.layers.Dense(units=10, activation='softmax')(net)
model = tf.keras.models.Model(inputs=net_in,outputs=net)


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

tensorboard_callback = TensorBoard(log_dir=LOG_DIR, 
                           histogram_freq=1,
                           write_graph=True,
                           write_images=True)

model.summary()

In [0]:
# model training and evaluation part
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

In [0]:
# prediction 

test_input = x_test[300,:,:]
pred_y = model.predict(test_input.reshape([1,28,28]))

plt.figure(1)
plt.imshow(test_input)
plt.title("input")
print("model prediction = %s"% pred_y.argmax())
